## Extract compositional information from a map of a porphyrtic lava

In [1]:
# import the chemMap package
import chemMap

# import common python packages for data processing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# turn off certain warnings and use the matplotlib backend: Qt5Agg to get interactive plots
pd.options.mode.chained_assignment = default='warn'
%matplotlib

# set basic plot parameters (or preferences)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 18})

Using matplotlib backend: Qt5Agg


First we need to define the elements that we are interested in and load them into a Python Dictionary. The loadcnt function will search for datafiles titled 'E K series.csv' in the specified folder (where E represents any of the specified elements).

By specifying 'filter = True' we apply a 3-by-3 median filter to the raw data. This helps to remove artifacts whilst maintaining boundaries.

Finally, the resolution, in μm, is also specified.

In [2]:
Elements = ['Fe','Mg','Ca','Ti','Cr','Al','Na','Si','K']
Flo09W = chemMap.loadcnt('Galapagos/',Elements, filter=True)

Resolution=11

Now that the raw data has been imported, we can carry out k-means cluster analysis to determine whether we can identify different mineral phases in our sample, and also isolate background regions, or areas of no data (so that they can be avoided in further analysis).

In this first example we split the pixels into 6 different clusters and plot a map of the results (we can also specify a narrower range of elements if we wish for only a select group to be used for the cluster analysis). By specifying 'ShowComp = True', we also ask the code to print the average count rates for each cluster

In [3]:
Elements = ['Fe','Mg','Ca','Ti','Cr','Al','Na','Si','K']
Flo09W = chemMap.Cluster(Flo09W, Elements, 5, Plot = True, ShowComp = True)

Cluster 0,Fe = 49.46
Cluster 0,Mg = 32.19
Cluster 0,Ca = 268.6
Cluster 0,Ti = 17.87
Cluster 0,Cr = 5.52
Cluster 0,Al = 479.28
Cluster 0,Na = 79.69
Cluster 0,Si = 959.45
Cluster 0,K = 22.86
Cluster 1,Fe = 3.85
Cluster 1,Mg = 0.71
Cluster 1,Ca = 1.16
Cluster 1,Ti = 1.17
Cluster 1,Cr = 0.33
Cluster 1,Al = 1.61
Cluster 1,Na = 0.21
Cluster 1,Si = 3.55
Cluster 1,K = 0.08
Cluster 2,Fe = 226.14
Cluster 2,Mg = 412.16
Cluster 2,Ca = 0.98
Cluster 2,Ti = 5.76
Cluster 2,Cr = 2.23
Cluster 2,Al = 1160.1
Cluster 2,Na = 1.6
Cluster 2,Si = 7.53
Cluster 2,K = 0.33
Cluster 3,Fe = 294.04
Cluster 3,Mg = 714.11
Cluster 3,Ca = 17.53
Cluster 3,Ti = 3.71
Cluster 3,Cr = 1.83
Cluster 3,Al = 15.54
Cluster 3,Na = 4.47
Cluster 3,Si = 668.91
Cluster 3,K = 1.5
Cluster 4,Fe = 111.24
Cluster 4,Mg = 153.51
Cluster 4,Ca = 292.09
Cluster 4,Ti = 25.52
Cluster 4,Cr = 6.62
Cluster 4,Al = 201.93
Cluster 4,Na = 36.08
Cluster 4,Si = 895.83
Cluster 4,K = 13.25


Using these average count rates, we can now name the different clusters according to the mineral phases that we expect (or know) to be present in our sample. In the case of this lava flow, we have olivine, an oxide phase, clinopyroxene and plagioclase. 

There are several ways that you can name the clusters. First, as shown here, the NameCluster function can be used as shown. Alternatively, the NameCluster function can be triggered in the main Cluster function. To do so, either put 'Name = True' above, where you will be prompted to enter the name of each cluster as the compositions are printed, or enter the names of the clusters as a list (i.e., 'Name = ['Plg', 'nan', 'Spl', 'Oli', 'Cpx']')

In [4]:
Flo09W = chemMap.NameCluster(Flo09W, Name = ['Plg', 'nan', 'Spl', 'Oli', 'Cpx'])

Once we have identified the different clusters, and named them, we can start to assess the chemical properties/distributions of the various minerals. One interesting possibility is using the size distribution of a particular mineral phase to isolate mineral sub-groups. Here we choose to focus on olivine, as the maps show that the individual olivine crystals are isolated from each other (i.e., surrounded by clinopyroxene or plagioclase). This is important as the code searchs for the number of neighbouring, or touching pixels with the same cluster classification and, as a result, an interconnected framework of small groundmass crystals would be characterised as one very large crystal!

The Size function used below creates a numpy array of the same size of those for each element. Nan values are applied to any pixel that is not classified as the chosen cluster and each pixel of that cluster is assigned a value equal to the sum of pixels in that area that are classified as the chose cluster and are connected.

In [5]:
Flo09W = chemMap.Size(Flo09W, Cluster = 'Oli')

For minerals like olivine, however, plotting raw count values isn't particulary useful. Luckily, work by Matthew Locke, a former PhD student at Cardiff University, on a whole host of mineral and metal standards determined the correction factors that need to be applied to convert raw count ratios into ratios that are commonly used to describe mineral compositions (e.g., Anorthite content, Ca/(Ca+Na); Mg#, Mg/(Mg+Fe)).

Using the correction factors provided in the Quack software (see github.com/ianthomas23/QACD-quack) we can convert the raw count values into these ratios. 

In [6]:
Quant_MgNo = chemMap.detRatio(Flo09W, Elements, corr = 'olivine', ratio = 'Mg#')

Quant_An = chemMap.detRatio(Flo09W, Elements, corr = 'feldspar', ratio = 'An')

# assign these individual numpy arrays to the main Python dictionary
Flo09W['Olivine_Mg#'] = Quant_MgNo
Flo09W['Plagioclase_An'] = Quant_An

C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:198: RuntimeWarning: divide by zero encountered in true_divide
  counts[ratio]=counts['Mg']/(counts['Mg']+counts['Fe'])
C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:198: RuntimeWarning: invalid value encountered in true_divide
  counts[ratio]=counts['Mg']/(counts['Mg']+counts['Fe'])
C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:127: RuntimeWarning: divide by zero encountered in true_divide
  norm[ox] = Data[ox]/sum_ox
C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:127: RuntimeWarning: invalid value encountered in true_divide
  norm[ox] = Data[ox]/sum_ox
C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:153: RuntimeWarning: invalid value encountered in add
  A = A + norm[el]*element_properties[el][2]
C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:154: RuntimeWarning: invalid value encountered in add
  Z = Z + norm[el]*element_properties[el][1]
C:\Users\mattg

Now that these ratios have been determined, we can start to assess the compositional distributions of the different mineral phases. First, lets plot up a compositional map of the Forsterite content of olivine and the Anorthite content of plagioclase.

In [7]:
f, b = chemMap.ElementMap(Flo09W, ['Olivine_Mg#','Plagioclase_An'], ColMap = ['Reds','Blues'], 
                        Resolution = Resolution, Bounds = [[0.65, 0.95],[0.4,0.8]], Cluster = ['Oli','Plg'])

We can also extract the quantitative mineral compositions. For example, here we plot the composition of all olivines with typical areas less than ~50,000 μm^2, and those with areas greater than this specified area.

In [8]:
Area = (50000)/(Resolution**2)

f, a = plt.subplots(2,1, figsize=(8,8))
a[0].hist(Flo09W['Olivine_Mg#'][(Flo09W['Cluster']=='Oli') & (Flo09W['Size']<Area)], 
          bins  = np.linspace(0.65,0.95,201), density = True, facecolor = [1,0.2,0.2])
a[0].set_xlim([0.65, 0.95])

a[1].hist(Flo09W['Olivine_Mg#'][(Flo09W['Cluster']=='Oli') & (Flo09W['Size']>Area)], 
          bins  = np.linspace(0.65,0.95,201), density = True, facecolor = [0.56,0,0])
a[1].set_xlim([0.65, 0.95])
a[1].set_xlabel('Forsterite content')

Text(0.5, 0, 'Forsterite content')